In [ ]:
import requests
import pandas as pd

def extract_hunter_leads(hunter_response):
    data = hunter_response.get("data", {})
    emails = data.get("emails", [])

    company_name = data.get("organization")
    website = data.get("domain")

    leads = []

    for e in emails:
        lead = {
            "company_name": company_name,
            "website": website,
            "full_name": f"{e.get('first_name', '')} {e.get('last_name', '')}".strip(),
            "email": e.get("value"),
            "position": e.get("position"),
            "seniority": e.get("seniority"),
            "department": e.get("department"),
            "linkedin_url": e.get("linkedin"),
            "twitter_url": e.get("twitter"),
            "hunter_confidence": e.get("confidence")
        }

        leads.append(lead)

    return leads

API_KEY = ""
BASE_URL = "https://api.hunter.io/v2"
WEBSITE = "ramsayhealth.com"

url = f"{BASE_URL}/domain-search"
params = {
    "domain": WEBSITE,
    "limit": 10,
    "api_key": API_KEY
}

response = requests.get(url, params=params)
data = response.json()

leads = extract_hunter_leads(data)

for lead in leads:
    print(lead)

df = pd.DataFrame(leads)
df.to_csv(f"{WEBSITE}.csv", index=False)

{'company_name': 'Ramsay Health Care', 'website': 'ramsayhealth.com', 'full_name': 'Jacqui Kemp', 'email': 'kempj@ramsayhealth.com', 'position': 'Assistant Director of Clinical Services', 'seniority': 'executive', 'department': 'health', 'linkedin_url': 'https://www.linkedin.com/in/jacqui-kemp-507879287', 'twitter_url': None, 'hunter_confidence': 99}
{'company_name': 'Ramsay Health Care', 'website': 'ramsayhealth.com', 'full_name': 'Jason Murphy', 'email': 'murphyj@ramsayhealth.com', 'position': 'Finance Director', 'seniority': 'executive', 'department': 'finance', 'linkedin_url': 'https://www.linkedin.com/in/murphynumbers', 'twitter_url': None, 'hunter_confidence': 99}
{'company_name': 'Ramsay Health Care', 'website': 'ramsayhealth.com', 'full_name': 'Shaune Gillespie', 'email': 'gillespies@ramsayhealth.com', 'position': 'CEO', 'seniority': 'executive', 'department': 'executive', 'linkedin_url': 'https://www.linkedin.com/in/shaune-gillespie-7081a467', 'twitter_url': None, 'hunter_conf

In [ ]:
import requests
import time

# List of API Keys to rotate through
API_KEYS = [
]

BASE_URL = "https://api.hunter.io/v2"

def email_finder_bulk(targets):
    results = []
    key_index = 0  # Start with the first key

    for person in targets:
        name = person.get('name')
        domain = person.get('domain')
        success = False

        while not success:
            if key_index >= len(API_KEYS):
                print("🚨 All API keys have failed or been exhausted!")
                return results

            current_key = API_KEYS[key_index]
            url = f"{BASE_URL}/email-finder"
            params = {
                "full_name": name,
                "domain": domain,
                "api_key": current_key
            }

            print(f"🔎 Searching: {name} @ {domain} (Using Key #{key_index + 1})")

            try:
                response = requests.get(url, params=params)
                res_data = response.json()

                # Check for Authentication or Rate Limit errors
                if "errors" in res_data:
                    error_code = res_data['errors'][0].get('code')
                    error_detail = res_data['errors'][0].get('details')

                    # 401: Invalid Key | 429: Rate Limit Reached
                    if error_code in [401, 429]:
                        print(f"⚠️ Key #{key_index + 1} failed ({error_detail}). Switching key...")
                        key_index += 1
                        continue # Retry the same person with the next key
                    else:
                        print(f"❌ Skipping {name}: {error_detail}")
                        break # Move to next person if it's a data error (e.g. not found)

                # Successful Data Extraction
                data = res_data.get('data', {})
                if data:
                    entry = {
                        "domain": data.get('domain'),
                        "fullname": f"{data.get('first_name', '')} {data.get('last_name', '')}".strip(),
                        "position": data.get('position'),
                        "email": data.get('email'),
                        "score": data.get('score'),
                        "twitter": data.get('twitter'),
                        "linkedin_url": data.get('linkedin_url'),
                        "phone_number": data.get('phone_number')
                    }
                    results.append(entry)

                success = True
                time.sleep(0.5) # Brief pause to be safe

            except Exception as e:
                print(f"⚠️ Connection error: {e}. Retrying...")
                time.sleep(2)

    return results

# --- Example Usage ---
input_list = [
  {"name":"Andy Schmeltz","domain":"csl.com"},
  {"name":"Bill Mezzanotte","domain":"csl.com"},
  {"name":"Dave Ross","domain":"csl.com"},
  {"name":"Greg Boss","domain":"csl.com"},
  {"name":"Hervé Gisserot","domain":"csl.com"},
  {"name":"Kate Priestman","domain":"csl.com"},
  {"name":"Ken Lim","domain":"csl.com"},
  {"name":"Mark Hill","domain":"csl.com"},
  {"name":"Mary Oates","domain":"csl.com"},
  {"name":"Roanne Parry","domain":"csl.com"},

  {"name":"Chris Wilks","domain":"sonichealthcare.com"},
  {"name":"Diane Harari","domain":"sonichealthcare.com"},
  {"name":"Dr Jim Newcombe","domain":"sonichealthcare.com"},
  {"name":"Dr Stephen Fairy","domain":"sonichealthcare.com"},
  {"name":"Evangelos Kotsopoulos","domain":"sonichealthcare.com"},
  {"name":"Jess Onus","domain":"sonichealthcare.com"},
  {"name":"Leisel Hickey","domain":"sonichealthcare.com"},
  {"name":"Maki Takken","domain":"sonichealthcare.com"},
  {"name":"Paul Alexander","domain":"sonichealthcare.com"},
  {"name":"Peter Joseph","domain":"sonichealthcare.com"},
  {"name":"Peter Savage","domain":"sonichealthcare.com"},
  {"name":"Virginia Re","domain":"sonichealthcare.com"},
  {"name":"Dr Cynric Temple-Camp","domain":"sonichealthcare.com"},
  {"name":"Dr Ged Foley","domain":"sonichealthcare.com"},
  {"name":"Dr Ian Clark","domain":"sonichealthcare.com"},
  {"name":"Dr Julian Adler","domain":"sonichealthcare.com"},

  {"name":"Alison Deans","domain":"ramsayhealth.com"},
  {"name":"Craig Drummond","domain":"ramsayhealth.com"},
  {"name":"David Thodey","domain":"ramsayhealth.com"},
  {"name":"Dr Claudia Süssmuth Dyckerhoff","domain":"ramsayhealth.com"},
  {"name":"Helen Kurincic","domain":"ramsayhealth.com"},
  {"name":"Karen Penrose","domain":"ramsayhealth.com"},
  {"name":"Michael Siddle","domain":"ramsayhealth.com"},
  {"name":"Steven Sargent","domain":"ramsayhealth.com"},
  {"name":"Anthony Neilson","domain":"ramsayhealth.com"},
  {"name":"Colleen Harris","domain":"ramsayhealth.com"},
  {"name":"Dr Brindan Suresh","domain":"ramsayhealth.com"},
  {"name":"Dr John Doulis","domain":"ramsayhealth.com"},
  {"name":"Henrietta Rowe","domain":"ramsayhealth.com"},
  {"name":"Natalie Davis","domain":"ramsayhealth.com"},
  {"name":"Andrew Coombs","domain":"ramsayhealth.com"},
  {"name":"Joe O'Connor","domain":"ramsayhealth.com"},
  {"name":"Nick Costa","domain":"ramsayhealth.com"},
  {"name":"Pascal Roché","domain":"ramsayhealth.com"},
  {"name":"Stuart Winters","domain":"ramsayhealth.com"},

  {"name":"Andrew Thomson","domain":"healius.com.au"},
  {"name":"Anthea Muir","domain":"healius.com.au"},
  {"name":"Arjun Narang","domain":"healius.com.au"},
  {"name":"Kylie Brown","domain":"healius.com.au"},
  {"name":"Paul Anderson","domain":"healius.com.au"},
  {"name":"Paula Bayliss","domain":"healius.com.au"},
  {"name":"Prasad Arav","domain":"healius.com.au"},
  {"name":"Puneet Nagi","domain":"healius.com.au"},
  {"name":"Rodney Rudling","domain":"healius.com.au"},

  {"name":"Craig White","domain":"integraldiagnostics.com.au"},
  {"name":"Ian Kadish","domain":"integraldiagnostics.com.au"},
  {"name":"Julian Gully","domain":"integraldiagnostics.com.au"},
  {"name":"Melissa Wong","domain":"integraldiagnostics.com.au"},
  {"name":"Robin Harle","domain":"integraldiagnostics.com.au"},
  {"name":"Tony Chen","domain":"integraldiagnostics.com.au"},

  {"name":"Anthony Friedli","domain":"clinicallabs.com.au"},
  {"name":"Chris Barnes","domain":"clinicallabs.com.au"},
  {"name":"Chris Brownlow","domain":"clinicallabs.com.au"},
  {"name":"Darren McKee","domain":"clinicallabs.com.au"},
  {"name":"Joe Geran","domain":"clinicallabs.com.au"},
  {"name":"Matthew Cordingley","domain":"clinicallabs.com.au"},
  {"name":"Melinda McGrath","domain":"clinicallabs.com.au"},
  {"name":"Sean Jackson","domain":"clinicallabs.com.au"},
  {"name":"Shae Seymour","domain":"clinicallabs.com.au"},
  {"name":"Tony Landgren","domain":"clinicallabs.com.au"},

  {"name":"Anthony Bishop","domain":"cochlear.com"},
  {"name":"Brian Kaplan","domain":"cochlear.com"},
  {"name":"Dean Phizacklea","domain":"cochlear.com"},
  {"name":"Dig Howitt","domain":"cochlear.com"},
  {"name":"Greg Bodkin","domain":"cochlear.com"},
  {"name":"Jan Janssen","domain":"cochlear.com"},
  {"name":"Jennifer Hornery","domain":"cochlear.com"},
  {"name":"Karen O’Driscoll","domain":"cochlear.com"},
  {"name":"Lisa Aubert","domain":"cochlear.com"},
  {"name":"Rob McGrory","domain":"cochlear.com"},
  {"name":"Sarah Thom","domain":"cochlear.com"},
  {"name":"Stu Sayers","domain":"cochlear.com"},

  {"name":"Alastair Robertson","domain":"resmed.com"},
  {"name":"Brett Sandercock","domain":"resmed.com"},
  {"name":"Carlos M. Nunez","domain":"resmed.com"},
  {"name":"Dawn Haake","domain":"resmed.com"},
  {"name":"Jim Ellis","domain":"resmed.com"},
  {"name":"Justin Leong","domain":"resmed.com"},
  {"name":"Katrin Pucknat","domain":"resmed.com"},
  {"name":"Kylie Canaday","domain":"resmed.com"},
  {"name":"Michael Farrell","domain":"resmed.com"},
  {"name":"Michael Fliss","domain":"resmed.com"},
  {"name":"Michael Rider","domain":"resmed.com"},
  {"name":"Salli Schwartz","domain":"resmed.com"},
  {"name":"Shane Azzi","domain":"resmed.com"},
  {"name":"Todd Friedman","domain":"resmed.com"},
  {"name":"Vered Keisar","domain":"resmed.com"},

  {"name":"Andrew Somervell","domain":"fphcare.com"},
  {"name":"Andy Niccol","domain":"fphcare.com"},
  {"name":"Brian Schultz","domain":"fphcare.com"},
  {"name":"Desh Edirisuriya","domain":"fphcare.com"},
  {"name":"Jonti Rhodes","domain":"fphcare.com"},
  {"name":"Justin Callahan","domain":"fphcare.com"},
  {"name":"Lewis Gradon","domain":"fphcare.com"},
  {"name":"Lyndal York","domain":"fphcare.com"},
  {"name":"Marcus Driller","domain":"fphcare.com"},
  {"name":"Nicholas Fourie","domain":"fphcare.com"},
  {"name":"Nicola Talbot","domain":"fphcare.com"},
  {"name":"Raelene Leonard","domain":"fphcare.com"},
  {"name":"Winston Fong","domain":"fphcare.com"},

  {"name":"Augusto M. Accorsi","domain":"ansell.com"},
  {"name":"Brian Montgomery","domain":"ansell.com"},
  {"name":"Charlotte Doll","domain":"ansell.com"},
  {"name":"Deanna Johnston","domain":"ansell.com"},
  {"name":"John Marsden","domain":"ansell.com"},
  {"name":"Neil Salmon","domain":"ansell.com"},
  {"name":"Philippe H. Rommel","domain":"ansell.com"},
  {"name":"Rob Hughes","domain":"ansell.com"},
  {"name":"Sean P. Sweeney","domain":"ansell.com"},

  {"name":"Vikesh Ramsunder","domain":"sigmahealthcare.com.au"},
  {"name":"Richard Murray","domain":"sigmahealthcare.com.au"},
  {"name":"Kara McGowan","domain":"sigmahealthcare.com.au"},
  {"name":"Gary Woodford","domain":"sigmahealthcare.com.au"},

  {"name":"Philippe Wolgen","domain":"clinuvel.com"},
  {"name":"Peter Vaughan","domain":"clinuvel.com"},
  {"name":"Dennis Wright","domain":"clinuvel.com"},
  {"name":"Emilie Rodenburger","domain":"clinuvel.com"},
  {"name":"Lachlan Hay","domain":"clinuvel.com"},
  {"name":"Linda Teng","domain":"clinuvel.com"},
  {"name":"Malcolm Bull","domain":"clinuvel.com"},
  {"name":"Antonella Colucci","domain":"clinuvel.com"},
  {"name":"David Solomon","domain":"clinuvel.com"},
  {"name":"Azza Hamila","domain":"clinuvel.com"},

  {"name":"Michael Kavanagh","domain":"nanosonics.com"},
  {"name":"Rod Lopez","domain":"nanosonics.com"},
  {"name":"Matthew Carbines","domain":"nanosonics.com"},
  {"name":"Kimberly Hill","domain":"nanosonics.com"},
  {"name":"Jodi Sampson","domain":"nanosonics.com"},
  {"name":"Sunny Pillai","domain":"nanosonics.com"},
  {"name":"Jason Burriss","domain":"nanosonics.com"},
  {"name":"Jon Burdach","domain":"nanosonics.com"},
  {"name":"Derek Minihane","domain":"nanosonics.com"},
  {"name":"Ronan Wright","domain":"nanosonics.com"},
  {"name":"Katsumi Maruyama","domain":"nanosonics.com"},
  {"name":"Gareth Daly","domain":"nanosonics.com"},
  {"name":"Bill Haydon","domain":"nanosonics.com"},

  {"name":"Graham Hodges","domain":"regis.com.au"},
  {"name":"Linda Mellors","domain":"regis.com.au"},
  {"name":"Imtiaz Bhayat","domain":"regis.com.au"},
  {"name":"Filomena Ciavarella","domain":"regis.com.au"},
  {"name":"Gregg Funston","domain":"regis.com.au"},
  {"name":"Jodie Haydon","domain":"regis.com.au"},
  {"name":"Malcolm Ross","domain":"regis.com.au"},
  {"name":"Rick Rostolis","domain":"regis.com.au"},
  {"name":"James Theofanis","domain":"regis.com.au"},
  {"name":"Georgia Willis","domain":"regis.com.au"},

  {"name":"Scott Scoullar","domain":"summerset.co.nz"},
  {"name":"Margaret Warrington","domain":"summerset.co.nz"},
  {"name":"Fay French","domain":"summerset.co.nz"},
  {"name":"Aaron Smail","domain":"summerset.co.nz"},
  {"name":"Dean Tallentire","domain":"summerset.co.nz"},
  {"name":"Eleanor Young","domain":"summerset.co.nz"},
  {"name":"Kay Brodie","domain":"summerset.co.nz"},
  {"name":"Chris Lokum","domain":"summerset.co.nz"},
  {"name":"Stewart Scott","domain":"summerset.co.nz"},
  {"name":"Robyn Gillespie","domain":"summerset.co.nz"},

  {"name":"Mike Wilkins AO","domain":"medibank.com.au"},
  {"name":"David Koczkar","domain":"medibank.com.au"},
  {"name":"Kylie Bishop","domain":"medibank.com.au"},
  {"name":"Tom Exton","domain":"medibank.com.au"},
  {"name":"Milosh Milisavljevic","domain":"medibank.com.au"},
  {"name":"Mei Ramsay","domain":"medibank.com.au"},
  {"name":"Rob Read","domain":"medibank.com.au"},
  {"name":"Mark Rogers","domain":"medibank.com.au"},
  {"name":"Meaghan Telford","domain":"medibank.com.au"},
  {"name":"Felicia Trewin","domain":"medibank.com.au"},

  {"name":"Nicholas Stone","domain":"bupa.com.au"},
  {"name":"Stuart Hooper","domain":"bupa.com.au"},
  {"name":"Kate Williams","domain":"bupa.com.au"},
  {"name":"Andrew Kinkade","domain":"bupa.com.au"},
  {"name":"Guneet Sawhney","domain":"bupa.com.au"},
  {"name":"Nic Woods","domain":"bupa.com.au"},
  {"name":"Jim Nolan","domain":"bupa.com.au"},
  {"name":"Roger Sharp","domain":"bupa.com.au"},
  {"name":"Maria Marinelli","domain":"bupa.com.au"}
]


final_results = email_finder_bulk(input_list)

# Print Summary
print(f"\n✅ Processed {len(final_results)} records.")
for item in final_results:
    print(item)

In [3]:
import pandas as pd
df = pd.DataFrame(final_results)
df.to_csv("new_leads.csv", index=False)
df

,domain,fullname,position,email,score,twitter,linkedin_url,phone_number
0,csl.com,Andy Schmeltz,None,andy.schmeltz@csl.com,97.0,None,None,None
1,csl.com,Bill Mezzanotte,None,bill.mezzanotte@csl.com,94.0,None,None,None
2,csl.com,Dave Ross,None,dave.ross@csl.com,98.0,None,None,None
3,csl.com,Greg Boss,"Executive Vice President, General Counsel",greg.boss@csl.com,98.0,None,https://www.linkedin.com/in/gregory-boss,None
4,csl.com,Hervé Gisserot,None,herve.gisserot@csl.com,49.0,None,None,None
...,...,...,...,...,...,...,...,...
180,bupa.com.au,Guneet Sawhney,None,guneet.sawhney@bupa.com.au,95.0,None,None,None
181,bupa.com.au,Nic Woods,None,nic.woods@bupa.com.au,99.0,None,None,None
182,bupa.com.au,Jim Nolan,None,jim.nolan@bupa.com.au,95.0,None,None,None
183,bupa.com.au,Roger Sharp,None,roger.sharp@bupa.com.au,95.0,None,None,None
